<a href="https://colab.research.google.com/github/chaiminwoo0223/Deep-Learning/blob/main/20%20-%20Learning_Rate_Decay.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader

# Hyperparameter

In [2]:
batch_size = 256
lr = 0.001
num_epoch = 10

# Data
## 1.Data Download

In [3]:
mnist_train = dset.MNIST("./", train=True, transform=transforms.ToTensor(), target_transform=None, download=True)
mnist_test = dset.MNIST("./", train=False, transform=transforms.ToTensor(), target_transform=None, download=True)

100%|██████████| 9912422/9912422 [00:00<00:00, 214296743.70it/s]

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 56791230.11it/s]


Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 68590300.38it/s]

Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 23665253.13it/s]


Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



## 2.Check Dataset

In [4]:
print(mnist_train.__getitem__(0)[0].size(), mnist_train.__len__())
print(mnist_test.__getitem__(0)[0].size(), mnist_test.__len__())

torch.Size([1, 28, 28]) 60000
torch.Size([1, 28, 28]) 10000


## 3.Set DataLoader

In [5]:
train_loader = DataLoader(mnist_train, batch_size=batch_size, shuffle=True, num_workers=2, drop_last=True)
test_loader = DataLoader(mnist_test, batch_size=batch_size, shuffle=False, num_workers=2, drop_last=True)

# CNN
## 1.Model

In [6]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer = nn.Sequential(
            nn.Conv2d(1, 16, 3, padding=1),  # 28 X 28
            nn.ReLU(),
            nn.Conv2d(16, 32, 3, padding=1), # 28 X 28
            nn.ReLU(),
            nn.MaxPool2d(2, 2),              # 14 X 14
            nn.Conv2d(32, 64, 3, padding=1), # 14 X 14
            nn.ReLU(),
            nn.MaxPool2d(2, 2),              #  7 X 7
        )
        self.fc_layer = nn.Sequential(
            nn.Linear(64*7*7, 100),
            nn.ReLU(),
            nn.Linear(100, 10)
        )
        
    def forward(self, x):
        out = self.layer(x)
        out = out.view(batch_size, -1)
        out = self.fc_layer(out)
        return out

## 2.Loss_Func & Optimizer

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [8]:
model = CNN().to(device)
loss_func = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=lr)

In [9]:
# 지정한 스텝 단위로 학습률에 감마를 곱해 학습률을 감소시킨다.
# scheduler = lr_scheduler.StepLR(optimizer, step_size=1, gamma= 0.99)       

# 매 epoch마다 학습률에 감마를 곱한다.
# scheduler = lr_scheduler.ExponentialLR(optimizer, gamma= 0.99)

# 지정한 스텝 지점마다 학습률에 감마를 곱한다.
# scheduler = lr_scheduler.MultiStepLR(optimizer, milestones=[10, 30, 80], gamma= 0.1)  

scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, threshold=1, patience=1, mode='min')
print(dir(scheduler))
print(dir(optimizer))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_init_is_better', '_reduce_lr', '_reset', 'best', 'cooldown', 'cooldown_counter', 'eps', 'factor', 'in_cooldown', 'is_better', 'last_epoch', 'load_state_dict', 'min_lrs', 'mode', 'mode_worse', 'num_bad_epochs', 'optimizer', 'patience', 'state_dict', 'step', 'threshold', 'threshold_mode', 'verbose']
['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclassh

# Train

In [10]:
for i in range(num_epoch):
    for j, [image, label] in enumerate(train_loader):
        x = image.to(device)
        y_= label.to(device)

        optimizer.zero_grad()
        output = model.forward(x)
        loss = loss_func(output, y_)
        loss.backward()
        optimizer.step()

    scheduler.step(loss)
    if i % 10 == 0:
        print(loss)
    print("Epoch: {}, Learning_Rate: {}".format(i, scheduler.optimizer.state_dict()['param_groups'][0]['lr']))

tensor(2.2976, device='cuda:0', grad_fn=<NllLossBackward0>)
Epoch: 0, Learning_Rate: 0.001
Epoch: 1, Learning_Rate: 0.0001
Epoch: 2, Learning_Rate: 0.0001
Epoch: 3, Learning_Rate: 1e-05
Epoch: 4, Learning_Rate: 1e-05
Epoch: 5, Learning_Rate: 1.0000000000000002e-06
Epoch: 6, Learning_Rate: 1.0000000000000002e-06
Epoch: 7, Learning_Rate: 1.0000000000000002e-07
Epoch: 8, Learning_Rate: 1.0000000000000002e-07
Epoch: 9, Learning_Rate: 1.0000000000000004e-08


# Test

In [11]:
correct = 0
total = 0

with torch.no_grad():
    for image, label in test_loader:
        x = image.to(device)
        y_= label.to(device)

        output = model.forward(x)
        _, output_index = torch.max(output, 1)
        total += label.size(0)
        correct += (output_index == y_).sum().float()
        
    print("Accuracy of Test Data: {}".format(100*correct/total))

Accuracy of Test Data: 9.57532024383545
